## Import Dependencies

In [ ]:
from sklearn import cluster
import pandas as pd
import numpy as np
from math import exp

In [ ]:
df = pd.read_csv('players_season_stats.csv')

In [ ]:
teams_df = pd.read_csv('teams_season_stats.csv')

In [ ]:
def get_possession_df(teams_df):
    teams_df['possession'] = teams_df['possession'] * 100
    games_pos_df = teams_df[['team', 'possession']]
    teams_pos_df = games_pos_df.groupby('team').agg(['sum', 'count']).reset_index()
    teams_pos_df.columns = ['team_name', 'sum', 'count']
    teams_pos_df['possession'] = teams_pos_df['sum'] / teams_pos_df['count']
    return teams_pos_df

In [ ]:
teams_pos_df = get_possession_df(teams_df)

## Build dataset

#### Take a look at the initial dataset

In [ ]:
df.head()

#### Filter by mins played

In [ ]:
MINIMUM_MINUTES_PLAYED = 300

In [ ]:
df = df[df['mins_played'] >= MINIMUM_MINUTES_PLAYED]

#### Names + id

In [ ]:
players_df = df[['name', 'id', 'team_name']]

#### Define stats for each position

In [ ]:
stats_dict = {}

In [ ]:
stats_dict['total_stats'] = [
    'goals', 'headed_goals', 'kicked_goals', 'game_winning_goals', 'pk_goals',
    'assists', 'game_winning_assist', 'shots', 'shots_on_goal', 'crosses',
    'passes', 'touches', 'interceptions', 'blocks', 'tackles', 'clears',
    'goal_mouth_blocks', 'fouls_committed', 'fouls_suffered',
    'shot_conversion', 'shot_accuracy', 'adj_tackles', 'adj_interceptions', 'adj_blocks'
]

stats_dict['F'] = [
    'goals', 'headed_goals', 'kicked_goals',
    'assists', 'shots', 'shots_on_goal', 'crosses',
    'passes', 'touches', 'fouls_suffered',
    'shot_conversion', 'shot_accuracy'
]

stats_dict['M'] = [
    'goals', 'kicked_goals',
    'assists', 'shots', 'shots_on_goal', 'crosses',
    'passes', 'touches',
    'goal_mouth_blocks', 'fouls_committed', 'fouls_suffered',
    'shot_conversion', 'shot_accuracy', 'adj_tackles', 'adj_interceptions'
]

stats_dict['D'] = [
    'goals', 'headed_goals', 'crosses',
    'passes', 'touches', 'clears','goal_mouth_blocks', 
    'fouls_committed','adj_tackles', 'adj_interceptions', 'adj_blocks'
]

#### Build stats

In [ ]:
def adjust_tackles(tack, pos):
    return tack * 2 / (1 + exp(-0.1 * (pos - 50)))

In [ ]:
def adjust_def_plays(df, teams_df, stats):
    merged_df = pd.merge(df, teams_df, how='left')
    for s in stats:
        merged_df['adj_{}'.format(s)] = merged_df.apply(
            lambda x: adjust_tackles(x[s], x['possession']), axis=1
        )
    return merged_df.fillna(0)

In [ ]:
df['shot_accuracy'] = round(100 * df['shots_on_goal'] / df['shots']).fillna(0)
df['shot_conversion'] = round(100 * df['goals'] / df['shots_on_goal']
                              ).fillna(0)

In [ ]:
adjusted_stats = ['tackles', 'interceptions', 'blocks']

In [ ]:
df = adjust_def_plays(df, teams_pos_df, adjusted_stats)

In [ ]:
stats_90 = [
        s for s in stats_dict['total_stats'] if s not in ['shot_conversion', 'shot_accuracy']
    ]

In [ ]:
for s in stats_90:
    df[s] = 90 * df[s] / df['mins_played']

In [ ]:
df.head()

#### Get dict with positions dfs

In [ ]:
df_dict = {}

for pos, pos_stats in stats_dict.items():
    if pos != 'total_stats':
        cols = ['id'] + pos_stats
        df_dict[pos] = df[df['position'] == pos][cols].set_index('id').fillna(0)

## Model

###### K-means

In [ ]:
k = 35

In [ ]:
algorithm = cluster.KMeans(n_clusters=k)

#### Models

In [ ]:
def run_models(df_dict, clf):
    results_dict = {}
    for pos, pos_df in df_dict.items():
        print('\nPosition: {}'.format(pos))
        mat = pos_df.values
        results_dict[pos] = {}
        clf.fit(mat)
        res_df = pd.DataFrame([pos_df.index, clf.labels_]).T
        res_df.columns = ['id', 'cat']
        final_df = pd.merge(res_df, players_df)
        results_dict[pos] = final_df
    return results_dict

In [ ]:
results_dict = run_models(df_dict, algorithm)

## Results

#### Specific Player

In [ ]:
def get_player_cluster(players):
    pos_cats = {}
    for pos, pos_df in results_dict.items():
        player_df = pos_df[pos_df['name'].isin(players)]
        if not player_df.empty:
            pos_cats[pos] = player_df['cat'].tolist()
    return pos_cats

In [ ]:
def print_sel_clusters(players, results_dict):
    pos_cats = get_player_cluster(players)
    for pos, cats in pos_cats.items():
        print('\nPosition: ' + pos)
        res_df = results_dict[pos]
        for cat in cats:
            print('\n', res_df[res_df['cat'] == cat])

In [ ]:
players_to_show = ['Nicolas Blandi', 'Lucas Pratto', 'Cristian Pavon', 'Ricardo Centurion', 'Matias Caruzzo',
                  'Leonardo Ponzio', 'Fernando Gago', 'Ignacio Scocco', 'Rodrigo Mora', 'Maximiliano Meza']

In [ ]:
print_sel_clusters(players_to_show, results_dict)